In [ ]:
# Imports

import sys
from cltkreaders.grc import GreekTesseraeCorpusReader
import unicodedata
import re
from natsort import natsorted
from pprint import pprint

In [ ]:
# Load corpus, get files
T = GreekTesseraeCorpusReader()
homer = [file for file in T.fileids() if 'homer' in file]

print(homer[:5])

In [ ]:
# Sort by book number

homer = natsorted(homer)
print(homer)

## Discovering 'clearchics'

from Athenaeus 11.87, tr. Yonge ([Perseus](http://www.perseus.tufts.edu/hopper/text?doc=Perseus%3Atext%3A2013.01.0003%3Abook%3D11%3Achapter%3D87))
>This, then, was what Clearchus said; and the things which he says one ought to propose, are, I imagine, such as these. For one person to quote a line in Homer beginning with Alpha, and ending with the same letter, such as...  
>  
> ἀγχοῦ δ᾽ ἱσταμένη ἔπεα πτερόεντα προσηύδα.  
> ἀλλ᾽ ἄγε νῦν μάστιγα καὶ ἡνία σιγαλόεντα.  
> ἀσπίδας εὐκύκλους λαισήαϊ τε πτερόεντα.  

In [ ]:
# Get citation and text per line

lines = []

for doc_row in T.doc_rows(homer):
    lines.extend(doc_row.items())

doc_rows = dict(lines)

citations, lines = zip(*lines)

print(citations[:5])
print(lines[:5])

In [ ]:
# Describe lines in Homer

print(f'There are {len(lines)} lines in Homer.')

In [ ]:
# Helper function to remove diacriticals

def remove_diacriticals(text):
    combining_character_table = dict.fromkeys(c for c in range(sys.maxunicode) if unicodedata.combining(chr(c)))
    text = unicodedata.normalize('NFD', text)
    text = text.translate(combining_character_table)
    return text

In [ ]:
# Helper function for preprocessing
def preprocess(text, lower=True, punctuation=False, numbers=False, remove_spaces=False, diacriticals=True):
    if lower:
        text = text.lower() # Lowercase

    if not punctuation:
        # Remove punctuation
        punctuation ="\"#$%&\'()*+,/:;<=>@[\]^_`{|}~.?!«»—“-”"
        misc = '¡£¤¥¦§¨©¯°±²³´µ¶·¸¹º¼½¾¿÷·–‘’†•ↄ∞⏑〈〉（）'
        misc += punctuation
        translator = str.maketrans({key: " " for key in misc})
        text = text.translate(translator)

    if not numbers:
        # Remove numbers
        translator = str.maketrans({key: " " for key in '0123456789'})
        text = text.translate(translator)

    if remove_spaces:
        text = "".join(text.split())

    if not diacriticals:
        text = remove_diacriticals(text)

    # Fix spacing
    text = re.sub(' +', ' ', text)

    text = unicodedata.normalize('NFC', text)

    return text.strip()

In [ ]:
# Preprocess lines, i.e. case, numbers, punctuation, etc.
prep_lines = [preprocess(line, diacriticals=True) for line in lines]
pprint(prep_lines[:5])

In [ ]:
# Remove diacriticals; fastest way

concatenated_lines = "\n".join(prep_lines)
concatenated_lines = preprocess(concatenated_lines, diacriticals=False)
prep_lines = concatenated_lines.split('\n')
print(len(prep_lines))

print(prep_lines[:5])

In [ ]:
# Check for clearchics

clearchics = []

for citation, line in zip(citations, prep_lines):
    if line[0] == line[-1]:
        clearchics.append(citation)

print(f'There are {len(clearchics)} clearchics in Homer.')

In [ ]:
import random

clearchic_sample = random.sample(clearchics, 10)

print('Here is a random sample of ten (10) clearchics from Homer...')

for i, clearchic in enumerate(clearchic_sample, 1):
    print(f'{i}: {clearchic}: {doc_rows[clearchic]}')

In [ ]:
# Note how above we made a dict with citations as keys and original lines as values to make look ups easier later

doc_rows['<hom.od 23.112>']